In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
SPPARKS ANISOTROPIC MODEL PREPROCESSING UTILITIES
=================================================

This notebook provides essential preprocessing tools for converting simulation data
between VECTOR analysis format and SPPARKS (Stochastic Parallel PARticle Kinetic 
Simulator) input format. SPPARKS is used for Monte Carlo simulations of grain growth
with anisotropic energy models.

Key Functionality:
1. Convert SPPARKS dump files to VECTOR-compatible init files
2. Extract Euler angle data from original initialization files  
3. Generate neighbor list files for 3D periodic boundary conditions
4. Support for anisotropic grain boundary energy models

Data Flow:
Original Init → SPPARKS Simulation → Dump Files → Processed Init → Neighbor Files

Created on Mon Jul 31 14:33:57 2023
@author: Lin
"""

import os
current_path = os.getcwd()
import numpy as np
from numpy import seterr
seterr(all='raise')  # Enable strict error handling for numerical operations
import matplotlib.pyplot as plt
import math
from tqdm import tqdm  # Progress bars for file processing operations
import multiprocess as mp  # Multiprocessing for neighbor list generation
import sys
sys.path.append(current_path+'/../../')

# VECTOR framework modules for grain boundary analysis
import myInput
import post_processing
import PACKAGE_MP_3DLinear as linear3d

import importlib
importlib.reload(post_processing)  # Reload for development/debugging

<module 'post_processing' from '/home/lin.yang/projects/VECTOR/examples/dump_to_init/../../post_processing.py'>

In [ ]:
def get_line(i, j):
    """
    Calculate row index for grain pair (i,j) in MisoEnergy.txt lookup table.
    
    SPPARKS uses symmetric grain boundary energy matrices stored as compressed
    upper triangular format. This function maps 2D grain pair indices to 1D
    storage index.
    
    Parameters:
    -----------
    i, j : int
        Grain IDs (i < j for proper symmetric storage)
        
    Returns:
    --------
    int : Row index in compressed energy matrix
    
    Mathematical Formula: 
    For symmetric matrix storage, row = i + (j-1)*j/2 when i < j
    """
    if i < j: 
        return i+(j-1)*(j)/2
    else: 
        return j+(i-1)*(i)/2

def init2EAarray(init_file_path, grain_num):
    """
    Extract Euler angles from SPPARKS initialization file into numpy array.
    
    SPPARKS init files contain crystallographic orientation data as Euler angles
    (phi1, Phi, phi2) in Bunge notation. This function parses the file format
    and creates a dense array for efficient access during processing.
    
    Parameters:
    -----------
    init_file_path : str
        Path to SPPARKS .init file containing grain orientations
    grain_num : int
        Expected number of grains in the microstructure
        
    Returns:
    --------
    numpy.ndarray : Shape (grain_num, 3) containing Euler angles
        Column 0: phi1 (rotation about Z-axis)
        Column 1: Phi  (rotation about X-axis)  
        Column 2: phi2 (rotation about Z-axis)
        
    File Format Expected:
    Line 0-2: Headers (ignored)
    Line 3+: site_id grain_id phi1 Phi phi2
    
    Notes:
    ------
    - Missing grains are filled with [0,0,0] orientation
    - Euler angles represent crystallographic orientation for anisotropy calculations
    """
    euler_angle_array = np.ones((grain_num, 3))*-2  # Initialize with sentinel values
    
    with open(init_file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file):
            # Skip header lines (first 3 lines)
            if i > 2: 
                data = line.split()
                grain_id = int(data[1]) - 1  # Convert to 0-based indexing
                euler_angles = np.array([float(data[2]), float(data[3]), float(data[4])])
                euler_angle_array[grain_id] = euler_angles
    
    # Check for missing grains and fill with default orientation
    for i in range(grain_num):
        if euler_angle_array[i,0] == -2:
            print(f"Warning: Missing grain {i+1}, using default orientation [0,0,0]")
            euler_angle_array[i] = np.array([0,0,0])
            
    return euler_angle_array

def output_init_from_dump(dump_file_path, euler_angle_array, init_file_path_output):
    """
    Convert SPPARKS dump file to initialization format with Euler angle mapping.
    
    SPPARKS dump files contain evolved microstructure states from Monte Carlo
    simulation. This function extracts the final grain structure and maps
    crystallographic orientations from the original initialization file.
    
    Parameters:
    -----------
    dump_file_path : str
        Path to SPPARKS .dump file from simulation timestep
    euler_angle_array : numpy.ndarray
        Pre-loaded Euler angles from original init file
    init_file_path_output : str
        Output path for new initialization file
        
    Returns:
    --------
    tuple : (box_size, entry_length)
        box_size : numpy.ndarray - Simulation domain dimensions [Lx, Ly, Lz]
        entry_length : int - Total lines per dump entry (sites + headers)
        
    Dump File Format:
    Line 0: ITEM: TIMESTEP
    Line 1: timestep_value
    Line 2: ITEM: NUMBER OF ATOMS  
    Line 3: num_sites
    Line 4: ITEM: BOX BOUNDS
    Line 5-7: xlo xhi, ylo yhi, zlo zhi
    Line 8: ITEM: ATOMS id grain_id ...
    Line 9+: site_data
    
    Output Init Format:
    Line 0: # This line is ignored
    Line 1: Values
    Line 2: [blank]
    Line 3+: site_id grain_id phi1 Phi phi2
    
    Notes:
    ------
    - Preserves crystallographic orientation during microstructure evolution
    - Supports 2D and 3D periodic domain geometries
    - Essential for anisotropic grain boundary energy calculations
    """
    # Parse dump file header information
    with open(dump_file_path) as file:
        box_size = np.zeros(3)
        for i, line in enumerate(file):
            if i==3: num_sites = int(line)
            if i==5: box_size[0] = np.array(line.split(), dtype=float)[-1]
            if i==6: box_size[1] = np.array(line.split(), dtype=float)[-1] 
            if i==7: box_size[2] = np.array(line.split(), dtype=float)[-1]
            if i==8: name_vars = line.split()[2:]  # Variable names
            if i>8: break
            
    box_size = np.ceil(box_size).astype(int)  # Convert to integer grid size
    entry_length = num_sites + 9  # Header lines + site data

    # Write initialization file header
    IC_header = []
    IC_header.append("# This line is ignored\n")
    IC_header.append("Values\n") 
    IC_header.append("\n")
    
    with open(init_file_path_output, 'w') as output_file:
        output_file.writelines(IC_header)

    # Process dump data and write with Euler angle mapping
    with open(init_file_path_output, 'a') as output_file:
        with open(dump_file_path) as file:
            for i, line in tqdm(enumerate(file), 
                              f"Converting SPPARKS dump to init format", 
                              total=entry_length):
                
                if i==1: 
                    time_step = int(float(line.split()[-1]))
                    
                atom_num = i - 9  # Site data starts at line 9
                if 0 <= atom_num < num_sites:
                    line_split = np.array(line.split(), dtype=float)
                    site_id = int(line_split[0])
                    grain_id = int(line_split[1]) - 1  # Convert to 0-based indexing
                    
                    # Map crystallographic orientation from original grain
                    phi1, Phi, phi2 = euler_angle_array[grain_id]
                    output_file.write(f"{site_id} {grain_id+1} {phi1} {Phi} {phi2}\n")

    return box_size, entry_length

def output_init_neighbor_from_init(interval, box_size, init_file_path_input, init_file_path_output):
    """
    Generate SPPARKS neighbor list file with periodic boundary conditions.
    
    SPPARKS requires explicit neighbor connectivity for Monte Carlo grain growth
    simulations. This function creates neighbor lists for 3D periodic domains
    with customizable interaction range for anisotropic grain boundary models.
    
    Parameters:
    -----------
    interval : int
        Neighbor interaction range (typically 5 for grain boundary calculations)
        Creates (2*interval+3)^3 - 1 total neighbors per site
    box_size : numpy.ndarray  
        Domain dimensions [Lx, Ly, Lz] from simulation
    init_file_path_input : str
        Input initialization file with grain orientations
    init_file_path_output : str
        Output neighbor file for SPPARKS simulation
        
    Returns:
    --------
    bool : True if neighbor file generation successful
    
    Neighbor File Format:
    Line 0: # This line is ignored
    Line 1: 3 dimension
    Line 2: {nei_num} max neighbors  
    Line 3: {total_sites} sites
    Line 4-6: Domain bounds xlo xhi, ylo yhi, zlo zhi
    Line 7: [blank]
    Line 8: Sites
    Line 9: [blank]
    Line 10+: site_id x_coord y_coord z_coord
    [...Sites section...]
    Line N: [blank]
    Line N+1: Neighbors
    Line N+2: [blank]  
    Line N+3+: site_id neighbor1 neighbor2 ... neighborM
    [...Neighbors section...]
    [Appended Values section from input file]
    
    Algorithm Details:
    -----------------
    1. Create 3D site coordinate mapping using row-major ordering
    2. Generate periodic neighbor offsets in cubic interaction volume
    3. Use multiprocessing for efficient neighbor list computation
    4. Apply periodic boundary conditions with modulo arithmetic
    5. Append original grain orientation data from input file
    
    Notes:
    ------
    - Supports 2D (Lz=1) and 3D periodic boundary conditions
    - Memory-efficient chunked processing for large domains  
    - Essential for accurate anisotropic grain boundary energy calculations
    - Neighbor range determines grain boundary interaction physics
    """
    nei_num = (2*interval+3)**3-1  # Total neighbors per site
    size_x, size_y, size_z = box_size
    
    # Create 3D site ID mapping matrix (row-major ordering)
    img = np.zeros((size_y, size_x, size_z))
    print(f"Generating site ID matrix for {size_x}x{size_y}x{size_z} domain...")
    
    for k in tqdm(range(size_z), desc="Building site matrix"): 
        for i in range(size_y):  
            for j in range(size_x):  
                # Row-major site numbering: site_id = k*Lx*Ly + i*Lx + j
                img[i,j,k] = int(k*size_x*size_y + i*size_x + j)

    # Write neighbor file header information
    IC_header = []
    IC_header.append("# This line is ignored\n")
    IC_header.append("3 dimension\n")
    IC_header.append(f"{nei_num} max neighbors\n")
    IC_header.append(f"{size_x*size_y*size_z} sites\n")
    IC_header.append(f"0 {size_x} xlo xhi\n")
    IC_header.append(f"0 {size_y} ylo yhi\n") 
    IC_header.append(f"0 {size_z} zlo zhi\n")
    IC_header.append("\n")
    IC_header.append("Sites\n")
    IC_header.append("\n")
    
    with open(init_file_path_output, 'w') as file:
        file.writelines(IC_header)

    # Write site coordinate information
    print("Writing site coordinates...")
    with open(init_file_path_output, 'a') as file:
        for k in tqdm(range(size_z), desc="Writing sites"):
            for i in range(size_y):
                for j in range(size_x):
                    site_id = int(img[i,j,k] + 1)  # 1-based indexing for SPPARKS
                    file.write(f"{site_id} {float(j)} {float(i)} {float(k)}\n")

    # Prepare neighbor section header
    neighbor_header = ["\n", "Neighbors\n", "\n"]
    with open(init_file_path_output, 'a') as file:
        file.writelines(neighbor_header)

    # Generate periodic neighbor offset coordinates
    # Creates cubic interaction volume: (2*interval+3)^3 surrounding each site
    offsets = np.array(np.meshgrid(
        np.arange(-(interval + 1), interval + 2),
        np.arange(-(interval + 1), interval + 2), 
        np.arange(-(interval + 1), interval + 2),
    )).T.reshape(-1, 3)
    
    # Remove self-interaction [0,0,0] offset
    offsets = offsets[np.any(offsets != 0, axis=1)]

    def process_chunk(start_k, end_k, file_name):
        """
        Multiprocessing worker function for neighbor list generation.
        
        Processes a Z-slice chunk of the 3D domain to generate neighbor
        connectivity with periodic boundary conditions.
        """
        max_length_neighbors = 0
        
        with open(file_name, 'w') as file:
            for k in tqdm(range(start_k, end_k), desc=f"Processing Z={start_k}-{end_k}"):
                for i in range(size_y):
                    for j in range(size_x):
                        site_id = int(img[i,j,k] + 1)
                        neighbor_line = f"{site_id} "
                        
                        # Calculate periodic neighbor indices with wraparound
                        neighbor_coords = (np.array([i, j, k]) + offsets) % np.array([size_y, size_x, size_z])
                        
                        # Extract neighbor site IDs using advanced indexing
                        neighbor_ids = img[neighbor_coords[:, 0], 
                                         neighbor_coords[:, 1], 
                                         neighbor_coords[:, 2]].astype('int')
                        
                        # Convert to 1-based indexing and format as string
                        neighbor_line += ' '.join(map(str, neighbor_ids + 1))
                        max_length_neighbors = max(max_length_neighbors, len(neighbor_line))
                        file.write(neighbor_line + "\n")
            file.write("\n")
            
        print(f"Max neighbor line length: {max_length_neighbors} characters")

    # Multiprocessed neighbor list generation for performance
    print("Generating neighbor connectivity with multiprocessing...")
    num_processes = mp.cpu_count()
    chunk_size = size_z // num_processes
    processes = []
    temp_files = []

    # Distribute Z-slices across CPU cores
    for p in range(num_processes):
        start_k = p * chunk_size
        end_k = (p + 1) * chunk_size if p != num_processes - 1 else size_z
        temp_file = f'{init_file_path_output}_temp_{p}.txt'
        temp_files.append(temp_file)
        
        process = mp.Process(target=process_chunk, args=(start_k, end_k, temp_file))
        processes.append(process)
        process.start()

    # Wait for all processes to complete
    for process in processes:
        process.join()

    # Concatenate temporary files and clean up
    print("Consolidating neighbor data...")
    with open(init_file_path_output, 'a') as outfile:
        for fname in tqdm(temp_files, desc="Merging temp files"):
            with open(fname) as infile:
                outfile.write(infile.read())
            os.remove(fname)  # Clean up temporary files

    # Append original grain orientation values from input file
    print("Appending grain orientation data...")
    with open(init_file_path_input, 'r') as f_read:
        values_section = f_read.readlines()
        
    with open(init_file_path_output, 'a') as file:
        # Skip header lines from input file, append only Values section
        file.writelines(values_section[1:])
        
    print("Neighbor file generation completed successfully!")
    return True

In [ ]:
if __name__ == '__main__':
    """
    MAIN EXECUTION BLOCK: SPPARKS DUMP TO INIT CONVERSION
    ====================================================
    
    This section demonstrates the complete workflow for converting SPPARKS
    simulation dump files back to initialization format with preserved
    crystallographic orientations for subsequent analysis or re-simulation.
    
    Workflow Steps:
    1. Load original Euler angle data from initial configuration
    2. Convert final simulation dump to init format with orientation mapping
    3. Generate neighbor connectivity file for anisotropic simulations
    
    File Naming Convention:
    - Original init: Contains initial grain orientations
    - Dump files: Evolved microstructure from SPPARKS simulation  
    - Output init: Converted final state with original orientations
    - Neighbor file: Connectivity data for anisotropic energy calculations
    """

    # =================================================================
    # FILE PATH CONFIGURATION
    # =================================================================
    
    # Simulation timestep for data extraction
    last_step = 5
    
    # High-performance computing cluster storage paths
    dump_file_foler = "/orange/michael.tonks/lin.yang/"
    # Alternative paths for different computing environments:
    # dump_file_foler = "/blue/michael.tonks/lin.yang/SPPARKS-VirtualIncEnergy/2d_poly_multiCoreCompare/"
    # dump_file_foler = "/Users/lin/projects/SPPARKS-AGG/examples/Test_SimplifyIncE/2d_triple/"
    
    # Simulation identifier with key parameters
    # Naming convention: p_ori_fully5d_aveE_f{factor}_t{temp}_{size}_{grains}_multiCore{cores}_J{job}_refer_{ref}_seed{seed}_kt{temp}
    # dump_file_name = f"p_ori_fully5d_aveE_f1.0_t1.0_150_1k_multiCore64_J1_refer_1_0_0_seed56689_kt1.95"
    
    # Initialization file storage and naming
    init_file_folder = dump_file_foler + "IC/"
    init_file_name = f"VoronoiIC_256_200g.init"                    # Input: Original microstructure
    init_file_name_final = f"VoronoiIC_256_200g_neighbors5.init"   # Output: With neighbor connectivity
    # init_file_name_final = f"poly_IC150_1k.{last_step}_neighbor5.init"  # Alternative naming
    
    # =================================================================
    # MICROSTRUCTURE PARAMETERS  
    # =================================================================
    
    # Total number of grains in the microstructure
    grain_num = 200
    
    # Load crystallographic orientations from original initialization file
    # This preserves the Euler angle assignments for anisotropy calculations
    # euler_angle_array = post_processing.init2EAarray(init_file_folder+init_file_name_original, grain_num)

    # =================================================================
    # DUMP FILE PROCESSING (Currently Commented Out)
    # =================================================================
    
    # Convert SPPARKS dump file to VECTOR-compatible init format
    # This step extracts the evolved grain structure and maps orientations
    # dump_file_name_0 = dump_file_foler+dump_file_name+f".dump.{int(last_step)}"
    # box_size, entry_length = post_processing.output_init_from_dump(
    #     dump_file_name_0, 
    #     euler_angle_array, 
    #     init_file_folder+init_file_name
    # )
    # size_x, size_y, size_z = box_size
    
    print("Dump file processing workflow configured.")
    print("Uncomment relevant sections to execute full conversion pipeline.")

In [ ]:
if __name__ == '__main__':
    """
    NEIGHBOR FILE GENERATION SECTION
    ================================
    
    This section generates SPPARKS neighbor connectivity files required for
    anisotropic grain boundary energy simulations. The neighbor file defines
    the spatial interaction network for Monte Carlo grain growth calculations.
    
    Key Parameters:
    - interval: Determines neighbor interaction range and physics accuracy
    - box_size: Must match the simulation domain dimensions exactly
    - multiprocessing: Enables efficient processing of large 3D domains
    """

    # =================================================================
    # NEIGHBOR CONNECTIVITY PARAMETERS
    # =================================================================
    
    # Neighbor interaction range for grain boundary calculations
    # interval = 5: Creates 10^3 - 1 = 999 neighbors per site (3D)
    # Larger intervals capture longer-range anisotropic interactions
    # but increase computational cost and memory requirements
    interval = 5
    
    # Domain geometry specification
    # For 2D simulations: box_size = [Lx, Ly, 1] 
    # For 3D simulations: box_size = [Lx, Ly, Lz]
    box_size = np.array([256, 256, 1])  # 2D domain: 256x256 with Lz=1
    
    print(f"Generating neighbor file for {box_size} domain with interval={interval}")
    print(f"Expected neighbors per site: {(2*interval+3)**len(box_size[box_size>1])-1}")
    
    # =================================================================
    # MULTIPROCESSING NEIGHBOR FILE GENERATION  
    # =================================================================
    
    # Generate neighbor connectivity using optimized multiprocessing algorithm
    # This function handles:
    # 1. Periodic boundary condition calculations
    # 2. Memory-efficient chunked processing 
    # 3. Parallel computation across CPU cores
    # 4. File I/O optimization for large datasets
    output_neighbr_init = post_processing.output_init_neighbor_from_init_mp(
        interval,                                    # Neighbor interaction range
        box_size,                                    # Domain dimensions [Lx, Ly, Lz]
        init_file_folder + init_file_name,          # Input: Grain orientation data
        init_file_folder + init_file_name_final     # Output: Complete neighbor file
    )
    
    # Alternative single-threaded version for debugging or small domains:
    # output_neighbr_init = output_init_neighbor_from_init(
    #     interval, 
    #     box_size, 
    #     init_file_folder + init_file_name, 
    #     init_file_folder + init_file_name_final + "_test"
    # )
    
    # =================================================================
    # COMPLETION STATUS AND NEXT STEPS
    # =================================================================
    
    if output_neighbr_init:
        print("✓ Neighbor file generation completed successfully!")
        print(f"✓ Output file: {init_file_folder + init_file_name_final}")
        print("\nNext steps:")
        print("1. Verify neighbor file format compatibility with SPPARKS")
        print("2. Configure anisotropic grain boundary energy parameters")  
        print("3. Run SPPARKS simulation with generated neighbor connectivity")
        print("4. Monitor grain growth evolution and boundary migration")
    else:
        print("✗ Neighbor file generation failed!")
        print("Check input file paths and domain size parameters.")

> img matrix start.
> img matrix end
> Sites start writing


100%|██████████| 1/1 [00:00<00:00, 10.37it/s]


> Sites end writing
> Neighbors start writing


100%|██████████| 2/2 [00:00<00:00, 57.95it/s]


The max length of neighbor data line is 857

...


The max length of neighbor data line is 1013
The max length of neighbor data line is 1013
The max length of neighbor data line is 1013


100%|██████████| 2/2 [00:00<00:00, 68.55it/s]


The max length of neighbor data line is 987


Concatenating : 100%|██████████| 128/128 [00:00<00:00, 529.56it/s]


> Neighbors end writing
> Values start writing
> Values read done
> Values end writing
